In [5]:
import tensorflow as tf
print(tf.__version__)

2.17.0


# 填充和步幅

在上一节的例子里，我们使用高和宽为3的输入与高和宽为2的卷积核得到高和宽为2的输出。一般来说，假设输入形状是$n_h\times n_w$，卷积核窗口形状是$k_h\times k_w$，那么输出形状将会是

$$(n_h-k_h+1) \times (n_w-k_w+1).$$

所以卷积层的输出形状由输入形状和卷积核窗口形状决定。本节我们将介绍卷积层的两个超参数，即填充和步幅。它们可以对给定形状的输入和卷积核改变输出形状。

## 5.2.1 padding

填充（padding）是指在输入高和宽的两侧填充元素（通常是0元素）。图5.2里我们在原输入高和宽的两侧分别添加了值为0的元素，使得输入高和宽从3变成了5，并导致输出高和宽由2增加到4。图5.2中的阴影部分为第一个输出元素及其计算所使用的输入和核数组元素：$0\times0+0\times1+0\times2+0\times3=0$。

![在输入的高和宽两侧分别填充了0元素的二维互相关计算](../img/conv-pad.svg)

一般来说，如果在高的两侧一共填充$p_h$行，在宽的两侧一共填充$p_w$列，那么输出形状将会是

$$(n_h-k_h+p_h+1)\times(n_w-k_w+p_w+1),$$

也就是说，输出的高和宽会分别增加$p_h$和$p_w$。

在很多情况下，我们会设置$p_h=k_h-1$和$p_w=k_w-1$来使输入和输出具有相同的高和宽。这样会方便在构造网络时推测每个层的输出形状。假设这里$k_h$是奇数，我们会在高的两侧分别填充$p_h/2$行。如果$k_h$是偶数，一种可能是在输入的顶端一侧填充$\lceil p_h/2\rceil$行，而在底端一侧填充$\lfloor p_h/2\rfloor$行。在宽的两侧填充同理。

卷积神经网络经常使用奇数高宽的卷积核，如1、3、5和7，所以两端上的填充个数相等。对任意的二维数组`X`，设它的第`i`行第`j`列的元素为`X[i,j]`。当两端上的填充个数相等，并使输入和输出具有相同的高和宽时，我们就知道输出`Y[i,j]`是由输入以`X[i,j]`为中心的窗口同卷积核进行互相关计算得到的。

下面的例子里我们创建一个高和宽为3的二维卷积层，然后设输入高和宽两侧的填充数分别为1。给定一个高和宽为8的输入，我们发现输出的高和宽也是8。



















这段代码展示了如何使用填充（padding）进行卷积操作。让我详细解释：

1. **定义辅助函数**：
```python
def comp_conv2d(conv2d, X):
    X = tf.reshape(X,(1,) + X.shape + (1,))
    Y = conv2d(X)
    return tf.reshape(Y,Y.shape[1:3])
```


功能说明：
- 输入处理：将2D输入转换为4D格式
- 执行卷积：应用传入的卷积层
- 输出处理：去除批次和通道维度，返回2D结果

2. **创建卷积层**：
```python
conv2d = tf.keras.layers.Conv2D(filters=1, 
                               kernel_size=3, 
                               padding='same')
```


参数说明：
- filters=1：输出一个特征图
- kernel_size=3：使用3×3的卷积核
- padding='same'：使用相同填充，保持输出尺寸与输入相同

3. **创建测试数据**：
```python
X = tf.random.uniform(shape=(8,8))
```


- 创建8×8的随机输入矩阵
- 值均匀分布在[0,1]之间

4. **应用卷积并查看形状**：
```python
comp_conv2d(conv2d,X).shape
```


- 执行卷积操作
- 返回输出的形状

这段代码的特点：
1. **维度处理**：
   - 自动处理维度转换
   - 简化了卷积操作的使用

2. **填充使用**：
   - 使用'same'填充
   - 保持输出大小与输入相同

3. **便捷性**：
   - 封装了复杂的维度处理
   - 提供了简单的接口


In [6]:
# 定义一个函数来执行卷积操作并返回结果
def comp_conv2d(conv2d, X):
    # 将输入X重塑为4D张量，形状为(1, height, width, 1)
    X = tf.reshape(X,(1,) + X.shape + (1,))
    # 应用卷积操作
    Y = conv2d(X)
    # 返回结果
    # 去掉批次维度和通道维度，只取第二和第三维度的值
    return tf.reshape(Y,Y.shape[1:3])

# 创建一个2D卷积层，使用3x3的卷积核和'same'填充
# 输入形状 = (样本数, 行数, 列数, 通道数)
conv2d = tf.keras.layers.Conv2D(filters = 1, 
                                kernel_size=3, 
                                padding='same')
# 创建一个8x8的随机输入张量
X = tf.random.uniform(shape=(8,8))
# 应用卷积操作并打印结果的形状
comp_conv2d(conv2d,X).shape

TensorShape([8, 8])

## 5.2.2 stride

在上一节里我们介绍了二维互相关运算。卷积窗口从输入数组的最左上方开始，按从左往右、从上往下的顺序，依次在输入数组上滑动。我们将每次滑动的行数和列数称为步幅（stride）。

目前我们看到的例子里，在高和宽两个方向上步幅均为1。我们也可以使用更大步幅。图5.3展示了在高上步幅为3、在宽上步幅为2的二维互相关运算。可以看到，输出第一列第二个元素时，卷积窗口向下滑动了3行，而在输出第一行第二个元素时卷积窗口向右滑动了2列。当卷积窗口在输入上再向右滑动2列时，由于输入元素无法填满窗口，无结果输出。图5.3中的阴影部分为输出元素及其计算所使用的输入和核数组元素：$0\times0+0\times1+1\times2+2\times3=8$、$0\times0+6\times1+0\times2+0\times3=6$。

![高和宽上步幅分别为3和2的二维互相关运算](../img/conv-stride.svg)

一般来说，当高上步幅为$s_h$，宽上步幅为$s_w$时，输出形状为

$$\lfloor(n_h-k_h+p_h+s_h)/s_h\rfloor \times \lfloor(n_w-k_w+p_w+s_w)/s_w\rfloor.$$

如果设置$p_h=k_h-1$和$p_w=k_w-1$，那么输出形状将简化为$\lfloor(n_h+s_h-1)/s_h\rfloor \times \lfloor(n_w+s_w-1)/s_w\rfloor$。更进一步，如果输入的高和宽能分别被高和宽上的步幅整除，那么输出形状将是$(n_h/s_h) \times (n_w/s_w)$。

下面我们令高和宽上的步幅均为2，从而使输入的高和宽减半。





















这段代码展示了如何使用步幅（stride）和填充（padding）的卷积操作。让我详细解释：

1. **创建卷积层**：
```python
conv2d = tf.keras.layers.Conv2D(filters=1, 
                               kernel_size=3, 
                               padding='same',
                               strides=2)
```


参数说明：
- filters=1：输出一个特征图
- kernel_size=3：使用3×3的卷积核
- padding='same'：使用相同填充
- strides=2：设置步幅为2（新增参数）

2. **应用卷积**：
```python
comp_conv2d(conv2d, X).shape
```


- 使用之前定义的comp_conv2d函数
- 对8×8的输入X进行卷积
- 返回输出形状

这段代码的特点：

1. **步幅的作用**：
   - 步幅=2意味着卷积核每次移动2个单位
   - 会降低输出特征图的空间维度
   - 输出大小将是输入的一半（在每个空间维度上）

2. **填充和步幅的组合**：
   - 'same'填充保持边界信息
   - 步幅=2实现下采样
   - 最终输出大小为4×4（原始8×8的一半）

3. **计算原理**：
   - 输出大小 = ⌈输入大小/步幅⌉
   - 例如：8/2 = 4


padding='same'与strides=2的组合作用详解
让我详细解释padding='same'和strides=2的关系：

1. **padding='same'的作用**：
- 目的是在输入周围添加填充，使得输出大小满足特定规则
- 计算公式：
  ```
  输出大小 = ⌈输入大小/步幅⌉
  ```
- 为了达到这个目标，padding='same'会：
  - 在需要时自动计算并添加适当的填充
  - 尽可能在两边均匀添加填充
  - 如果需要不均匀填充，右边/底部会多一个填充

2. **strides=2对padding的影响**：
- 对于8×8的输入和3×3的卷积核：
  - 当strides=1时，需要填充1个像素在每边
  - 当strides=2时，需要填充1个像素在每边，但：
    - 右边和底部可能需要额外填充
    - 这是为了确保最后一个步幅能够完整覆盖输入

3. **具体例子**：
```
输入大小：8×8
卷积核：3×3
步幅：2
期望输出大小：4×4（8/2=4）

填充计算：
- 需要确保最后一个3×3窗口能完整覆盖
- 可能需要不对称填充
```

4. **padding='same'的工作方式**：
```python
# 对于步幅为2的情况
padding_needed = max(0, (output_size - 1) * strides + kernel_size - input_size)
padding_left = padding_needed // 2
padding_right = padding_needed - padding_left
```

5. **实际效果**：
- **确保输出大小严格等于⌈输入大小/步幅⌉**
- 保持特征图的空间信息
- 避免信息的不均匀丢失

6. **使用建议**：
- 当需要精确控制输出大小时使用
- 在构建深层网络时很有用
- 有助于保持特征图的对称性

总结：
- padding='same'和strides=2的组合确保了：
  - 合理的下采样比例
  - 特征信息的均匀保留
  - 可预测的输出大小

In [7]:
conv2d = tf.keras.layers.Conv2D(filters=1, 
                                kernel_size=3, 
                                padding='same',
                                strides=2)
comp_conv2d(conv2d, X).shape

TensorShape([4, 4])

接下来是一个稍微复杂点儿的例子。





















这段代码展示了使用非对称卷积核、非对称步幅和无填充的卷积操作。让我详细解释：

1. **创建卷积层**：
```python
conv2d = tf.keras.layers.Conv2D(filters=1, 
                               kernel_size=(3,5), 
                               padding='valid', 
                               strides=(3,4))
```



参数说明：
- filters=1：输出一个特征图
- kernel_size=(3,5)：使用3×5的非对称卷积核
- padding='valid'：不使用填充（默认值）
- strides=(3,4)：行方向步幅为3，列方向步幅为4

2. **查看和应用**：
```python
print(X.shape)  # 显示输入形状（8×8）
comp_conv2d(conv2d, X).shape  # 显示输出形状
```



输出尺寸计算：
- 对于'valid'填充（无填充）：
  - 输出高度 = ⌊(8-3)/3⌋ + 1
  - 输出宽度 = ⌊(8-5)/4⌋ + 1

这段代码的特点：

1. **非对称设计**：
   - 卷积核在高度和宽度上不同
   - 步幅在两个方向上不同

2. **无填充效果**：
   - padding='valid'意味着不添加填充
   - 输出尺寸会显著减小

3. **尺寸变化**：
   - 输入：8×8
   - 输出尺寸会根据公式计算
   - 体现了不同参数对输出大小的影响

这种设置适用于：
- 需要不同方向上不同程度的特征提取
- 需要快速降低特征图尺寸
- 特征具有方向性的场景

In [9]:
conv2d = tf.keras.layers.Conv2D(filters=1, 
                                kernel_size=(3,5), 
                                padding='valid', 
                                strides=(3,4))
print(X.shape)
comp_conv2d(conv2d, X).shape


(8, 8)


TensorShape([2, 1])